# This code was modified and tested in google colab.

In [1]:
!pip install tensorflow_datasets

In [2]:
!pip install matplotlib

In [3]:
!pip install -q tensorflow_model_optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 4.0 MB/s eta 0:00:00


In [4]:
!pip install tf.keras

In [5]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
import tempfile
tf.random.set_seed(
    seed=81
)

In [6]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [7]:
#recreating saved model
keras_file = './Full_Precision_MNIST_TF.h5'
model = tf.keras.models.load_model(keras_file)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 6)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 16)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 120)               9

In [8]:
_, baseline_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_accuracy)

Baseline test accuracy: 0.9850999712944031


In [9]:
#pruning...

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

#computing end step
batch_size = 128
n_epochs = 2
val_split = 0.1

num_images = train_images.shape[0] * (1 - val_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * n_epochs

#defining pruning model
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

#need to recompile for low magnitude pruning
model_for_pruning.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 28, 28, 6)         308       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 14, 14, 6)         1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_conv2d  (None, 14, 14, 16)        4818      
 _1 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_po  (None, 7, 7, 16)          1         
 oling2d_1 (PruneLowMagnitu                                      
 de)                                                    

In [11]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=n_epochs,
                  callbacks=callbacks)

Epoch 1/2
469/469 [==============================] - 37s 78ms/step - loss: 0.0379 - accuracy: 0.9881
Epoch 2/2
469/469 [==============================] - 37s 80ms/step - loss: 0.0332 - accuracy: 0.9898


In [12]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9850999712944031
Pruned test accuracy: 0.9876999855041504


In [13]:
##docs_infra: no_execute
#%tensorboard --logdir={logdir}

In [14]:
#creating compressible model for tensorflow
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

#creating compressible model for TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

<ipython-input-14-8c4e35b7c35f>:5: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: /tmp/tmppvjtrj7f.h5
Saved pruned TFLite model to: /tmp/tmpnfcb3kke.tflite


In [15]:
#helper function to compress models via gzip
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [16]:
#comparing model sizes

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 1200897.00 bytes
Size of gzipped pruned Keras model: 131939.00 bytes
Size of gzipped pruned TFlite model: 127641.00 bytes


In [22]:
#applying post-training quantization to pruned model
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

#_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

f = open("quantized_and_pruned_tflite_file", "wb") #w for write
f.write(quantized_and_pruned_tflite_model)
f.close()


print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size("quantized_and_pruned_tflite_file")))

Size of gzipped baseline Keras model: 1200897.00 bytes
Size of gzipped pruned and quantized TFlite model: 40933.00 bytes


In [18]:
#helper function to evaluate TFLite model
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)

    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy


In [19]:
#evaluating pruned and quantized model

#interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
#interpreter.allocate_tensors()

#test_accuracy = evaluate_model(interpreter)

#print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
#print('Pruned TF test accuracy:', model_for_pruning_accuracy)

In [23]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat /content/quantized_and_pruned_tflite_file | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 709,842 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
